# Zillow Home Value Index Prediction

This project was a part of an assignment of the CIVE 7100: Time Series and Geospatial Data Sciences Course at Northeastern University.

The assignment was kind of like a Kaggle Competition wherein, data from Zillow HVI data for the United States was given. The task was to implement a model that most accurately predicts the missing ZHVI values in the dataset.

Grade Received: 100/100



## Importing Libraries and Dataset

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score, mean_squared_error

In [2]:
df = pd.read_csv('homework_df.csv')
df

,RegionName,StateName,county_geoid,year,zhvi,median_income,lon,lat,row_id
0,Autauga County,AL,1001,2012,175393.416667,53773.0,-86.642740,32.534890,1
1,Autauga County,AL,1001,2013,173808.166667,53682.0,-86.642740,32.534890,2
2,Autauga County,AL,1001,2014,176132.250000,52475.0,-86.642740,32.534890,3
3,Autauga County,AL,1001,2015,181524.166667,51281.0,-86.642740,32.534890,4
4,Autauga County,AL,1001,2016,186105.583333,53099.0,-86.642740,32.534890,5
...,...,...,...,...,...,...,...,...,...
22571,Weston County,WY,56045,2015,162598.500000,57738.0,-104.567859,43.840219,22572
22572,Weston County,WY,56045,2016,NaN,55640.0,-104.567859,43.840219,22573
22573,Weston County,WY,56045,2017,169520.750000,59605.0,-104.567859,43.840219,22574
22574,Weston County,WY,56045,2018,175115.916667,52867.0,-104.567859,43.840219,22575


Our test dataset is the rows in which the values for `zhvi` are missing.

In [3]:
missing_original = df[df['zhvi'].isna()]
missing_original

,RegionName,StateName,county_geoid,year,zhvi,median_income,lon,lat,row_id
7,Autauga County,AL,1001,2019,NaN,58731.0,-86.642740,32.534890,8
16,Barbour County,AL,1005,2012,NaN,31889.0,-85.393408,31.869526,17
17,Barbour County,AL,1005,2013,NaN,32911.0,-85.393408,31.869526,18
18,Barbour County,AL,1005,2014,NaN,35634.0,-85.393408,31.869526,19
19,Barbour County,AL,1005,2015,NaN,32964.0,-85.393408,31.869526,20
...,...,...,...,...,...,...,...,...,...
22555,Uinta County,WY,56041,2015,NaN,56569.0,-110.547573,41.287415,22556
22558,Uinta County,WY,56041,2018,NaN,58235.0,-110.547573,41.287415,22559
22562,Washakie County,WY,56043,2014,NaN,45696.0,-107.681879,43.905102,22563
22568,Weston County,WY,56045,2012,NaN,53802.0,-104.567859,43.840219,22569


## Exploring the dataset

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22576 entries, 0 to 22575
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   RegionName     22576 non-null  object 
 1   StateName      22576 non-null  object 
 2   county_geoid   22576 non-null  int64  
 3   year           22576 non-null  int64  
 4   zhvi           16618 non-null  float64
 5   median_income  22575 non-null  float64
 6   lon            22576 non-null  float64
 7   lat            22576 non-null  float64
 8   row_id         22576 non-null  int64  
dtypes: float64(4), int64(3), object(2)
memory usage: 1.6+ MB


## One hot encoding

In [5]:
#creating instance of one-hot-encoder
encoder = OneHotEncoder(handle_unknown='ignore')

#perform one-hot encoding on 'team' column 
encoder_df = pd.DataFrame(encoder.fit_transform(df[['StateName', 'RegionName']]).toarray())
encoder_df

,0,1,2,3,4,5,6,7,8,9,...,1753,1754,1755,1756,1757,1758,1759,1760,1761,1762
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22572,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22573,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22574,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
#merge one-hot encoded columns back with original DataFrame
new_df = df.join(encoder_df)
df = new_df.drop(['StateName', 'RegionName'], axis=1)
df

,county_geoid,year,zhvi,median_income,lon,lat,row_id,0,1,2,...,1753,1754,1755,1756,1757,1758,1759,1760,1761,1762
0,1001,2012,175393.416667,53773.0,-86.642740,32.534890,1,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1001,2013,173808.166667,53682.0,-86.642740,32.534890,2,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1001,2014,176132.250000,52475.0,-86.642740,32.534890,3,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1001,2015,181524.166667,51281.0,-86.642740,32.534890,4,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1001,2016,186105.583333,53099.0,-86.642740,32.534890,5,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22571,56045,2015,162598.500000,57738.0,-104.567859,43.840219,22572,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22572,56045,2016,NaN,55640.0,-104.567859,43.840219,22573,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22573,56045,2017,169520.750000,59605.0,-104.567859,43.840219,22574,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22574,56045,2018,175115.916667,52867.0,-104.567859,43.840219,22575,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
missing = df[df['zhvi'].isna()]
missing

,county_geoid,year,zhvi,median_income,lon,lat,row_id,0,1,2,...,1753,1754,1755,1756,1757,1758,1759,1760,1761,1762
7,1001,2019,NaN,58731.0,-86.642740,32.534890,8,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,1005,2012,NaN,31889.0,-85.393408,31.869526,17,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,1005,2013,NaN,32911.0,-85.393408,31.869526,18,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18,1005,2014,NaN,35634.0,-85.393408,31.869526,19,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19,1005,2015,NaN,32964.0,-85.393408,31.869526,20,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22555,56041,2015,NaN,56569.0,-110.547573,41.287415,22556,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22558,56041,2018,NaN,58235.0,-110.547573,41.287415,22559,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22562,56043,2014,NaN,45696.0,-107.681879,43.905102,22563,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22568,56045,2012,NaN,53802.0,-104.567859,43.840219,22569,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
nonmissing = df[~df['zhvi'].isna()]
nonmissing

,county_geoid,year,zhvi,median_income,lon,lat,row_id,0,1,2,...,1753,1754,1755,1756,1757,1758,1759,1760,1761,1762
0,1001,2012,175393.416667,53773.0,-86.642740,32.534890,1,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1001,2013,173808.166667,53682.0,-86.642740,32.534890,2,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1001,2014,176132.250000,52475.0,-86.642740,32.534890,3,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1001,2015,181524.166667,51281.0,-86.642740,32.534890,4,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1001,2016,186105.583333,53099.0,-86.642740,32.534890,5,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22570,56045,2014,157310.666667,53920.0,-104.567859,43.840219,22571,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22571,56045,2015,162598.500000,57738.0,-104.567859,43.840219,22572,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22573,56045,2017,169520.750000,59605.0,-104.567859,43.840219,22574,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22574,56045,2018,175115.916667,52867.0,-104.567859,43.840219,22575,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
nonmissing.fillna(method='ffill', inplace=True)

C:\Users\kedar\anaconda3\lib\site-packages\pandas\core\frame.py:5176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


## Splitting the data

In [10]:
X = nonmissing.drop(labels=['county_geoid', 'lon', 'lat','row_id', 'zhvi'], axis=1)
y = nonmissing['zhvi']

In [11]:
X = X.reset_index().drop(['index'], axis=1)
X

,year,median_income,0,1,2,3,4,5,6,7,...,1753,1754,1755,1756,1757,1758,1759,1760,1761,1762
0,2012,53773.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2013,53682.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2014,52475.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2015,51281.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2016,53099.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16613,2014,53920.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16614,2015,57738.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16615,2017,59605.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16616,2018,52867.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
X

,year,median_income,0,1,2,3,4,5,6,7,...,1753,1754,1755,1756,1757,1758,1759,1760,1761,1762
0,2012,53773.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2013,53682.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2014,52475.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2015,51281.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2016,53099.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16613,2014,53920.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16614,2015,57738.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16615,2017,59605.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16616,2018,52867.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=420)

## Using Ridge Regression

In [14]:
model = Ridge()
model.fit(X_train, y_train)

C:\Users\kedar\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


Ridge()

In [15]:
predictions = model.predict(X_test)

C:\Users\kedar\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


## Evaluation

In [16]:
r2_score(y_test, predictions)

0.8713751999635275

In [17]:
#RMSE
np.sqrt(mean_squared_error(y_test, predictions))

35567.02716759922

In [18]:
# Bias
bias = np.mean(predictions-y_test)
bias

1094.6733302197506

In [19]:
100*np.mean((predictions-y_test) / y_test)

3.527351703169378

In [20]:
np.quantile(predictions, 0.99) - np.quantile(y_test, 0.99)

-85880.55161421216

In [21]:
y_test

15207     84517.750000
20982     54505.833333
343       49723.083333
9265      87585.333333
10372    102744.166667
             ...      
13982     70272.166667
6028     140366.000000
5971      92692.916667
15118    166928.500000
11440    149139.166667
Name: zhvi, Length: 4986, dtype: float64

In [22]:
predictions

array([ 92393.22868787,  73512.8964521 ,  83956.39691707, ...,
        89194.73354182, 152666.84128461, 146748.56332923])

In [23]:
missing_new = missing.drop(labels=['county_geoid', 'lon', 'lat','row_id', 'zhvi'], axis=1)
missing_new = missing_new.reset_index().drop(['index'], axis=1)
missing_new

,year,median_income,0,1,2,3,4,5,6,7,...,1753,1754,1755,1756,1757,1758,1759,1760,1761,1762
0,2019,58731.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2012,31889.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2013,32911.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2014,35634.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2015,32964.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5953,2015,56569.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5954,2018,58235.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5955,2014,45696.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5956,2012,53802.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
test_predictions = model.predict(missing_new)

C:\Users\kedar\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [25]:
test_predictions

array([205165.83870837,  40790.18580517,  47079.87293229, ...,
       149251.95649046, 162889.57050967, 177739.35918528])

In [26]:
missing_original['yhat'] = test_predictions
missing_original

C:\Users\kedar\AppData\Local\Temp/ipykernel_12804/1312865450.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_original['yhat'] = test_predictions


,RegionName,StateName,county_geoid,year,zhvi,median_income,lon,lat,row_id,yhat
7,Autauga County,AL,1001,2019,NaN,58731.0,-86.642740,32.534890,8,205165.838708
16,Barbour County,AL,1005,2012,NaN,31889.0,-85.393408,31.869526,17,40790.185805
17,Barbour County,AL,1005,2013,NaN,32911.0,-85.393408,31.869526,18,47079.872932
18,Barbour County,AL,1005,2014,NaN,35634.0,-85.393408,31.869526,19,61163.133693
19,Barbour County,AL,1005,2015,NaN,32964.0,-85.393408,31.869526,20,50536.963174
...,...,...,...,...,...,...,...,...,...,...
22555,Uinta County,WY,56041,2015,NaN,56569.0,-110.547573,41.287415,22556,199993.603899
22558,Uinta County,WY,56041,2018,NaN,58235.0,-110.547573,41.287415,22559,212448.201384
22562,Washakie County,WY,56043,2014,NaN,45696.0,-107.681879,43.905102,22563,149251.956490
22568,Weston County,WY,56045,2012,NaN,53802.0,-104.567859,43.840219,22569,162889.570510


In [27]:
missing_original.to_csv('Kedar_Ghule_Homework.csv', index=False)